In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
%matplotlib inline
from keras.layers import Input, Dense
from keras.models import Model
# from keras.op
from keras import optimizers
from keras import regularizers
from PIL import Image
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.coordinates import FK5
from astropy import wcs
from astropy.io import fits
import sys
from astropy import units as u
from astropy.coordinates import SkyCoord

In [76]:
data = pd.read_csv("./DFBS.csv")

In [93]:
data.head(20)
print(data.shape)

(7358, 9)


In [78]:
all_hdr_files = []
listOfFiles = os.listdir('./fits headers')
pattern = "*.hdr"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            all_hdr_files.append('./fits headers/'+entry)

In [79]:
print(len(all_hdr_files))

1700


In [80]:
#hdulist = fits.open(all_hdr_files[0])

In [81]:
#w = wcs.WCS(hdulist[0].header)

In [82]:
#pixcrd = np.array([[0, 0], [24, 38], [4801 * 2, 4701 * 2]], dtype=np.float64)
#world = w.wcs_pix2world(pixcrd, 0)

#print(world)

In [83]:
#w

In [84]:
#c = SkyCoord(100.174423, -55.203358, frame='icrs', unit='deg')

In [100]:
#dx, dy = w.wcs_world2pix(100.174423, -55.203358, 0)
#print(dx, dy)

In [92]:
data["plate"] = np.nan

In [94]:
data["dx"] = np.zeros(data.shape[0])

In [95]:
data["dy"] = np.zeros(data.shape[0])

In [96]:
data.head()

,_Glon,_Glat,_RAJ2000,_DEJ2000,Cl,Name,Vmag,z,plate,dx,dy
0,deg,deg,deg,deg,NaN,NaN,mag,NaN,NaN,0.0,0.0
1,100.174423,-55.203358,0.04875,5.388056,Sy1,RXS J00001+0523,16.4,0.040,NaN,0.0,0.0
2,99.844434,-57.30727,0.61,3.351667,Sy1,MARK 543,14.68,0.026,NaN,0.0,0.0
3,86.112841,-70.112882,0.88375,-10.744722,Sy1,NGC 7808,15.4,0.029,NaN,0.0,0.0
4,114.304767,-16.638006,1.039583,45.440278,Sy1,RXS J00041+4526,16.9,0.120,NaN,0.0,0.0


In [ ]:
def isNaN(x):
    return str(x) == str(1e400*0)
count = 0
for index, row in data.iterrows():
    if index == 0: continue
    lon = float(row["_Glon"])
    lat = float(row["_Glat"])
    #print(lon, lat)
    for i in range(len(all_hdr_files)):
        hdulist = fits.open(all_hdr_files[i])
        w = wcs.WCS(hdulist[0].header)
        dx, dy = w.wcs_world2pix(lon, lat, 1)
        if not(isNaN(dx)) and not(isNaN(dy)) and dx >= 0 and dy >= 0 and dx < 10000 and dy < 10000:
            print(dx, dy)
            count += 1
        #else:
        #    print("no")
    
    break

In [121]:
hdulist[0].header

SIMPLE  =                    T                                                  
BITPIX  =                    8 / Array data type                                
NAXIS   =                    0                                                  
EXTEND  =                    T                                                  
        -------------------- Original data of observation                       
DATEORIG= '1975-05-02'         / Original recorded date of the observation      
TMS-ORIG= 'UT 00:00'           / Start of the observation (logs)                
TIMEFLAG= 'missing '           / Quality flag of the recorded observation time  
RA-ORIG = '12:11:05.999'       / RA of plate center as given in source          
DEC-ORIG= '+81:51:39.96'       / Dec of plate center as given in source         
OBJECT  = 'FBS 1254'           / Observed object or field                       
OBJTYPE = 'field   '           / Object type as in WFPDB                        
EXPTIME =                 12